In [1]:
from threading import local


def str_list_to_list_int(x):
    res = x.strip('][').split(', ')
    res = [int(i) for i in res]
    return res

In [2]:
def str_list_to_list(x):
    res = x.strip('][').split(', ')
    return res

In [3]:
def duration_to_second(x):
    x = str_list_to_list_int(x)
    lst = []
    for i in x:
        lst.append(i / 1000)
    return lst

In [4]:
import pandas as pd
train_df = pd.read_csv("train")

In [81]:
#test
# import pandas as pd
# test_df = pd.read_parquet("test.parquet")
#
# test_df['item_id'] = test_df['item_id'].apply(str_list_to_list)

In [83]:
# for i, row in test_df.iterrows():
#     row.values[1] = row.values[1][0]


KeyboardInterrupt: 

In [5]:
train_df['watch_time'] = train_df['watch_time'].apply(str_list_to_list_int)

In [6]:
train_df['item_id'] = train_df['item_id'].apply(str_list_to_list)

In [7]:
train_df['duration'] = train_df['duration'].apply(duration_to_second)

In [8]:
def calc_rait(row):
    lst = []
    for i in range(len(row.watch_time)):
        lst.append(round(row.watch_time[i] / (row.duration[i] + 0.001), 3))
    return lst

In [9]:
train_df['raiting'] = train_df.apply(lambda row: calc_rait(row), axis=1)
train_df

,user_id,item_id,watch_time,duration,raiting
0,user_10000000,['video_1487101'],[2310],[5377.48],[0.43]
1,user_10000099,['video_856096'],[172],[176.36],[0.975]
2,user_1000020,['video_547304'],[902],[2941.0],[0.307]
3,user_10000345,"['video_1393364', 'video_2084529', 'video_4947...","[359, 399, 244, 458, 247, 653]","[373.889, 450.731, 273.409, 498.369, 303.723, ...","[0.96, 0.885, 0.892, 0.919, 0.813, 0.994]"
4,user_10000388,['video_283933'],[1823],[5616.28],[0.325]
...,...,...,...,...,...
140104,user_9998487,['video_1893827'],[1589],[4276.8],[0.372]
140105,user_9998803,['video_984416'],[1990],[6365.68],[0.313]
140106,user_9999389,['video_1250322'],[2421],[2437.203],[0.993]
140107,user_9999509,['video_1409879'],[3583],[4362.658],[0.821]


In [10]:
df = train_df[['user_id','item_id','raiting']]
df

,user_id,item_id,raiting
0,user_10000000,['video_1487101'],[0.43]
1,user_10000099,['video_856096'],[0.975]
2,user_1000020,['video_547304'],[0.307]
3,user_10000345,"['video_1393364', 'video_2084529', 'video_4947...","[0.96, 0.885, 0.892, 0.919, 0.813, 0.994]"
4,user_10000388,['video_283933'],[0.325]
...,...,...,...
140104,user_9998487,['video_1893827'],[0.372]
140105,user_9998803,['video_984416'],[0.313]
140106,user_9999389,['video_1250322'],[0.993]
140107,user_9999509,['video_1409879'],[0.821]


In [11]:
clean_data = []
for i, row in df.iterrows():
    if len(row.values[1]) >= len(row.values[2]):
        length = len(row.values[1])
    else:
        length = len(row.values[2])
    for j in range(length):
        clean_data.append([pd.to_numeric(row.values[0].strip("user_")), pd.to_numeric(row.values[1][j].strip("'").strip("video_")), pd.to_numeric(row.values[2][j])])

In [12]:
clean_data_df = pd.DataFrame(clean_data, columns=['user_id','item_id','raiting'])


In [13]:
clean_data_df

,user_id,item_id,raiting
0,10000000,1487101,0.430
1,10000099,856096,0.975
2,1000020,547304,0.307
3,10000345,1393364,0.960
4,10000345,2084529,0.885
...,...,...,...
396153,9998487,1893827,0.372
396154,9998803,984416,0.313
396155,9999389,1250322,0.993
396156,9999509,1409879,0.821


In [14]:
clean_data_df.to_csv("clean_data_df.csv", index=False)

In [23]:
from pyspark.sql.functions import col, explode
from pyspark import SparkContext

In [24]:
from pyspark.sql import SparkSession
sc = SparkContext
# sc.setCheckpointDir('checkpoint')
spark = SparkSession.builder.appName('Recommendations').getOrCreate()

In [25]:
ratings = spark.createDataFrame(clean_data_df)

In [26]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [27]:
# Create test and train set
(train, test) = ratings.randomSplit([0.8, 0.2], seed = 1234)

# Create ALS model
als = ALS(userCol="user_id", itemCol="item_id", ratingCol="raiting", nonnegative = True, implicitPrefs = False, coldStartStrategy="drop")

# Confirm that a model called "als" was created
type(als)

pyspark.ml.recommendation.ALS

In [28]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Add hyperparameters and their respective values to param_grid
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100, 150]) \
            .addGrid(als.regParam, [.01, .05, .1, .15]) \
            .build()
            #             .addGrid(als.maxIter, [5, 50, 100, 200]) \


# Define evaluator as RMSE and print length of evaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="raiting", predictionCol="prediction")
print ("Num models to be tested: ", len(param_grid))

Num models to be tested:  16


In [29]:
# Build cross validation using CrossValidator
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

# Confirm cv was built
print(cv)

CrossValidator_2c79e3fd2ec8


In [ ]:
#Fit cross validator to the 'train' dataset
model = cv.fit(train)

#Extract best model from the cv model above
best_model = model.bestModel